<a href="https://colab.research.google.com/github/JarikDev/home_work_4/blob/main/%D0%93%D0%B0%D1%82%D0%B8%D0%BD%D1%81%D0%BA%D0%B8%D0%B9_%D0%AF_%D0%90_%D0%94%D0%97_PYTHON_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# старт таймера
start_time = time.time()
desired_width = 320
pd.set_option('display.width', desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option('display.max_columns', 20)


# функция замеряет время от переданного аргументом старта
def return_execution_time(start):
    return time.time() - start


# класс, который поможет нам сделать базовую аналитику по датафреймам
class DataFrameAnalyzer:
    def __init__(self, *dataframes):
        self.__dataframes = dataframes

    # статический метод описывает инфо по датафрейму
    @staticmethod
    def __show_df_info(df):
        print(f"Датафрейм {df.name} содержит {len(df.columns)} колонок")
        print(f"Датафрейм {df.name} содержит следующие колонки {df.columns.values}")
        print(f"Подробная информация по датафрейму {df.name}")
        print(df.info())

    # метод возвращает первые пять строк датафрейма
    @staticmethod
    def __show_df_heads(df):
        print(f"Первые пять строк датафрейма {df.name}")
        print(df.head())

    # метод показывает форма датафрейма
    @staticmethod
    def __show_df_shapes(df):
        print(f"Датафрейм {df.name} имеет форму {df.shape}")

    # метод описывает описательные характеристики датафрейма
    @staticmethod
    def __show_df_described_info(df):
        print(f"Данные датафрейма {df.name} имеют следующие характеристики:")
        described_info = df.describe()
        # берём колонки
        columns = described_info.columns.values
        # итерируется по колонкам и выводим информацию
        for c in columns:
            print(f"Для колонки {c} количество непустых строк {described_info[c].count()}")
            print(f"Для колонки {c} общее количество записей {described_info[c].mean()}")
            print(f"Для колонки {c} стандартное отклонение {described_info[c].std()}")
            q25 = described_info[c].loc["25%"]
            print(f"Для колонки {c} общее количество записей в первый квартиль {q25}")
            q50 = described_info[c].loc["50%"]
            print(f"Для колонки {c} общее количество записей в третий квартиль {q50}")
            q75 = described_info[c].loc["75%"]
            print(f"Для колонки {c} общее количество записей попавших в четвёртый квартиль {q75}")
            print(f"Для колонки {c} минимальное значение {described_info[c].min()}")
            print(f"Для колонки {c} максимальное значение {described_info[c].max()}")

        print(f"Данные датафрейма {df.name} колонки с типом object имеют следующие характеристики:")
        described_info = df.describe(include=['object'])
        # берём колонки с типом обжэект и выводим информацию
        columns = described_info.columns.values
        for c in columns:
            print(f"Для колонки {c} количество непустых строк {described_info[c]['count']}")
            print(f"Для колонки {c} количество уникальных значений {described_info[c]['unique']}")
            print(f"Для колонки {c} самое частое значение - мода {described_info[c].top}")
            print(f"Для колонки {c} частота  {described_info[c].freq}")

    # метод описывает количественные характеристики уникальных полей датафрейма
    @staticmethod
    def __show_df_unique_values(df):
        print(f"Проведём анализ уникальных значений в датафрейме {df.name}")
        columns = df.columns.values
        for c in columns:
            uniques = df[c].value_counts()
            for k, v in uniques.items():
                print(f"В колонке {c} значения {k} встречается {v} раз")

    # метод выводит пропуски в данных, если они есть
    @staticmethod
    def __show_df_empties(df):
        print(f"Проведём анализ пустых значений в датафрейме {df.name}")
        print(df.isnull().sum())

    # метод проводит подробный анализ датафрейма
    def perform_analysis(self):
        for df in self.__dataframes:
            print(f"##### Анализ датафрейма {df.name} #####")
            self.__show_df_info(df)
            self.__show_df_shapes(df)
            self.__show_df_heads(df)
            self.__show_df_empties(df)
            self.__show_df_described_info(df)
            # next operation is very slow.
            # self.__show_df_unique_values(df)


# загружаем датафреймы и присваиваем им имена
ld = pd.read_csv('/content/drive/MyDrive/hw4/c_lectures.csv', sep=',')
ld.name = "lectures_data"
qd = pd.read_csv('/content/drive/MyDrive/hw4/c_questions.csv', sep=',')
qd.name = "questions_data"
td = pd.read_csv('/content/drive/MyDrive/hw4/c_train.csv', sep=',')
td.name = "train_data"
analyzer = DataFrameAnalyzer(ld, qd, td)
# проводим анализ датафреймов
analyzer.perform_analysis()

# заполняем пропуски в датафреймах
qd["tags"].replace([0], np.nan)
td_min_prior_question_elapsed_time = td["prior_question_elapsed_time"].min()
td["prior_question_elapsed_time"] = td["prior_question_elapsed_time"].astype('float64')
td["prior_question_elapsed_time"] = td["prior_question_elapsed_time"].replace(np.nan,
                                                                              td_min_prior_question_elapsed_time)
td["prior_question_had_explanation"] = td["prior_question_had_explanation"].astype('bool')

# собираем всё в один датафрейм
merged_df = pd.merge(td, qd, left_on='content_id', right_on='question_id')
merged_df = pd.merge(merged_df, ld, left_on='content_id', right_on='lecture_id')
merged_df.name = "merged_df"
analyzer = DataFrameAnalyzer(merged_df)
# проводим анализ получившегося датафрейма
analyzer.perform_analysis()

# 1. анализируем успеваемость студентов
# 1 - ответ верный, 0 - нет, -1 - лекция
# создаём себе вспомогательные кортежи для фильтрации датафреймов
is_question_answer = (merged_df['answered_correctly'] == 1)
is_correct = (merged_df['answered_correctly'] == 1)
is_incorrect = (merged_df['answered_correctly'] == 0)
is_lecture = (merged_df['answered_correctly'] == -1)
is_question = (merged_df['answered_correctly'] != -1)

# собираем статистику по ответам пользователя
user_answer_stat = pd.DataFrame({"user_id": merged_df["user_id"].unique()})

# собираем уникальные айди пользователя
user_ids = merged_df["user_id"].unique()
# собираем данные о правильных ответах
correct_answers = merged_df[is_correct][["user_id", "answered_correctly"]].groupby("user_id").count()
# собираем данные об ответах всего
users_answers_total = merged_df[is_question][["user_id", "answered_correctly"]].groupby("user_id").count()
# собираем данные об ответах пользователя в один датафрейм
answer_stat = pd.DataFrame({"user_id": user_ids})
answer_stat = pd.merge(answer_stat, correct_answers, left_on='user_id', right_on='user_id')
answer_stat = pd.merge(answer_stat, users_answers_total, left_on='user_id', right_on='user_id')
answer_stat.dropna(how='all')
# вычисляем процент правильныз ответов
answer_stat['correct_answers_percent'] = (answer_stat["answered_correctly_x"] / answer_stat[
    "answered_correctly_y"]) * 100
# сортируем результат для большей наглядности
answer_stat = answer_stat.sort_values('correct_answers_percent')
# в качестве выводов выводим таблицу с результатами
print("""
        Выводы:
        Ниже таблица с процентом правильных ответов для каждого студента.""")
print(answer_stat)

# 2. Анализируем зависимость правильности ответа от времени на него затраченного
# собираем данные по правильным ответам и затраченному на каждый ответ времени. Разбиваем на группы.
cut_time_elapsed_for_correct_answers = pd.cut(merged_df[is_correct]['prior_question_elapsed_time'], 50,
                                              labels=range(1, 51), ordered=False)
# подсчитываем вхождения в каждую группу
cut_time_elapsed_for_correct_answers = cut_time_elapsed_for_correct_answers.value_counts()
# сортируем по индексу
cut_time_elapsed_for_correct_answers = cut_time_elapsed_for_correct_answers.sort_index()
# выводим результат
print(cut_time_elapsed_for_correct_answers)
# Собираем данные по неправильным ответам и затраченному на каждый ответ времени. Разбиваем на группы.
cut_time_elapsed_for_incorrect_answers = pd.cut(merged_df[is_incorrect]['prior_question_elapsed_time'], 50,
                                                labels=range(1, 51), ordered=False)
# подсчитываем вхождения в каждую группу
cut_time_elapsed_for_incorrect_answers = cut_time_elapsed_for_incorrect_answers.value_counts()
# сортируем по индексу
cut_time_elapsed_for_incorrect_answers = cut_time_elapsed_for_incorrect_answers.sort_index()
# выводим результат
print(cut_time_elapsed_for_incorrect_answers)
# делаем выводы
print("""
        Выводы:
        Время затраченное на ответ до некоторого порога может повысить качество ответов. Поэтому спешить с ответом не стоит.
        Однако сидеть слишком долго желаемого результата не принесёт.
        Хотя у некоторого количества студентов, затративших большое количество времени на ответ удалось дать ответ правильный.
        Анализ неправильных ответов показывает, что большинство ответов даются относительно быстро, а дополнительное время затраченное
        на ответ сильно качество не повышает.
        Те кто думает над ответом максимальное время вдвое чаще отвечают правильно, чем неправильно.""")

# 3. Успеваемость по объяснениям (prior_question_had_explanation):
#   - Построить сводную таблицу с группировкой по наличию объяснений и посчитать средний процент успешных ответов.
# Собираем усреднённые данные по правильным и неправильным ответам
explanation_answer_data = merged_df[is_correct | is_incorrect].groupby('prior_question_had_explanation')[
                              'answered_correctly'].mean() * 100
# выводим результат
print(explanation_answer_data)
# делаем выводы
print(f"""
        Выводы:
        Процент правильных ответов среди тех которым предшествовали пояснения {explanation_answer_data[True]} %
        Процент правильных ответов среди тех которым пояснения не предшествовали {explanation_answer_data[False]} %
        Процент правильных ответов среди тех которым предшествовали пояснения выше.
        Пояснения повышают процент правильных ответов.""")

# 4. Связь вопросов и тегов (tags):
#   - Исследовать успеваемость по разным группам вопросов, выделенным по тегам (например, tags и part).
# Собираем данные по успеваемости студентов в разбивке по тегам
academic_performance_by_tag_data = merged_df[is_correct | is_incorrect].groupby('tags')[
                                       'answered_correctly'].mean() * 100
# сортируем по значениям дял наглядности
academic_performance_by_tag_data = academic_performance_by_tag_data.sort_values()
# выводим результат
print(academic_performance_by_tag_data)
# делаем выводы
print(f"""
        Выводы:
        Лучшая успеваемость у тем под тегами {academic_performance_by_tag_data.idxmax()}
        Худшая успеваемость у тем под тегами {academic_performance_by_tag_data.idxmin()}""")
# Собираем данные по успеваемости студентов в разбивке по частям
academic_performance_by_part_data = merged_df[is_correct | is_incorrect].groupby('part_x')[
                                        'answered_correctly'].mean() * 100
# сортируем по значениям дял наглядности
academic_performance_by_part_data = academic_performance_by_part_data.sort_values()
# выводим результат
print(academic_performance_by_part_data)
# делаем выводы
print(f"""
        Выводы:
        Лучшая успеваемость у тем части {academic_performance_by_part_data.idxmax()}
        Худшая успеваемость у тем части {academic_performance_by_part_data.idxmin()}""")

# 5. Анализ лекций (lectures.csv):
#   - Исследовать, как просмотр лекций и их категории влияют на результаты студентов.
# Собираем усреднённые данные о правильных и неправильны ответах студентов посмотревших лекции
lecture_influence_data = merged_df[is_correct | is_incorrect].groupby('type_of')[
                             'answered_correctly'].mean() * 100
# сортируем по значениям дял наглядности
lecture_influence_data = lecture_influence_data.sort_values()
# выводим результат
print(lecture_influence_data)
# делаем выводы
print(f"""
        Выводы:
        Полезнее всего просмотр лекций категории {lecture_influence_data.idxmax()}
        Менее всего полезен просмотр лекций категории {lecture_influence_data.idxmin()}""")

# 6. Динамика успеваемости студентов:
#   - Построим график распределение правильных ответов в зависимости от затраченного на ответ времени.
# Разбиваем данные на сегменты
time_elapsed_for_correct_answers = pd.cut(merged_df[is_correct]['prior_question_elapsed_time'], 100,
                                          labels=range(1, 101), ordered=False)
# Подсчитываем вхождения в каждый сегмент
time_elapsed_for_correct_answers = time_elapsed_for_correct_answers.value_counts()
# Сортируем по индексам
time_elapsed_for_correct_answers = time_elapsed_for_correct_answers.sort_index()
# Превращаем обратно в датафрейм
time_elapsed_for_correct_answers_df = time_elapsed_for_correct_answers.to_frame()
# Строим график
print(time_elapsed_for_correct_answers_df)
sns.displot(time_elapsed_for_correct_answers_df, x='count', kde=True)
plt.title('Распределение правильных ответов в зависимости от затраченного на ответ времени')
plt.show()

print(f"""
        Время выполнения {return_execution_time(start_time)} сек""")
